![logo](https://github.com/donatellacea/DL_tutorials/blob/main/notebooks/figures/1128-191-max.png?raw=true)

# Model-Agnostic Interpretation with Permutation Feature Importance

In this Notebook we will demonstrate how to use the Permutation Feature Importance method and interpret its results.

--------


### Setup Colab environment

If you installed the packages and requirments on your own machine, you can skip this section and start from the import section.
Otherwise you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

<a href="https://colab.research.google.com/github/HelmholtzAI-Consultants-Munich/Zero2Hero---Introduction-to-XAI/blob/master/xai-model-agnostic/tutorial_permutation_FI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


After the first execution you might receive some warning and notification, please follow these instructions:  

1. Warning: This notebook was not authored by Google.*  Click on 'Run anyway'.
2. Permit this notebook to access your Google Drive files? Click on 'Yes', and select your account.
3. Google Drive for desktop wants to access your Google Account. Click on 'Allow'.

At this point, a folder has been created and you can navigate it through the lefthand panel in Colab, you might also have received an email that informs you about the access on your Google Drive.

In [ ]:
# Create a folder in your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive

In [ ]:
# Don't run this cell if you already cloned the repo
!git clone https://github.com/HelmholtzAI-Consultants-Munich/Zero2Hero---Introduction-to-XAI.git  

In [ ]:
%cd Zero2Hero---Introduction-to-XAI/xai-model-agnostic

### Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils.utils import * 

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import get_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.inspection import permutation_importance

We fix the random seeds to ensure reproducible results, as we work with (pseudo) random numbers.

In [ ]:
# assert reproducible random number generation
seed = 1
np.random.seed(seed)

## The California Housing Dataset: Data Loading and Model Training

Let's use the California housing data set. The data pertains to the house prices found in a given California district and some summary statistics about them based on the 1990 census data.

<center><img src="./figures/california_housing.jpg" width="1200" /></center>

<font size=1> Source:\
https://www.kaggle.com/datasets/harrywang/housing

In [ ]:
calif_house_data = fetch_california_housing()

Now, let's have a look at the description of the dataset:

In [ ]:
print(calif_house_data['DESCR'])

Let's put all of our data into a data frame:

In [ ]:
X = pd.DataFrame(calif_house_data['data'], columns = calif_house_data['feature_names'])
y = pd.DataFrame(calif_house_data['target'], columns=['Price'])

See how many samples and features we have in the dataset and how their values are distributed:

In [ ]:
print('Features: {}'.format(list(X.columns)))
print('# samples: {}, # features: {}'.format(len(X.index), len(X.columns)))

In [ ]:
X.head()

In [ ]:
X.describe()

In [ ]:
y.head()

In [ ]:
y.describe()

The data set has 20640 observations, 8 features, and 1 target variable. **Note:** each row in the dataset represents a block of houses, not a single household.  
The target variable *price* is continuous and given in 100.000 $, so we can predict it using the other available features with a regression model of our choice.

For the sake of runtime we limit ourselves to only the first 2000 samples.

In [ ]:
X = X[:2000]
y = y[:2000]

We will split parts of the data, so the model can not use all the available information for training.  
That way, we can also check performance and interpretation results on previously unseen data, mirroring the most probable practical use case.

In [ ]:
# split off the test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=seed)
print('Number of training samples: {}'.format(len(X_train.index)))
print('Number of training samples: {}'.format(len(X_test.index)))

We'll create a Kernel ridge regression model with an RBF kernel to predict our *price* target variable from the other features, after they have been properly rescaled to each have zero mean and unit standard deviation.  
Don't worry for now if you are not familiar with the model. It is just meant as a protoype of a model that is not straightforward to interpret.

In [ ]:
pipe = Pipeline([
    ("preprocessing", StandardScaler()),
    ("model", KernelRidge(kernel="rbf"))
])

pipe.fit(X_train, y_train)

Now that the model is fit, let's check its performance as measured by the $R^2$ metric for both the training and the validation data. $R^2$ is the coefficient of determination and the closer this value is to 1, the better our model explains the data. A constant model that always predicts the average target value disregarding the input features would get an $R^2$ score of 0.  
**Note:** $R^2$ score can also be negative because the model can be arbitrarily worse.

In [ ]:
# is the model performing reasonably on the training data?
print('Model Performance on training data: {}'.format(pipe.score(X_train, y_train)))

# is the model performing reasonably on the test data?
print('Model Performance on test data: {}'.format(pipe.score(X_test, y_test)))

Since model training and tuning is not a focus of this course, we will not try to improve the model performance further.  
**Note:** you should keep in mind that interpreting a low performing model can lead to wrong conclusions.

Now that we trained a regression model which predicts the prices relatively well but might be a bit hard to interpret directly, we can make use of permutation feature importance to help us out.

## Now, what does my model actually think is important in the data?

### Permutation Feature Importance

We will try to get insights into which features are important by carrying out a method called **Permutation feature importance**. The following explanation of the approach is taken directly from the [scikit_learn documentation](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance):

"*Permutation feature importance is a model inspection technique that can be used for any fitted estimator when the data is tabular. This is especially useful for non-linear or opaque estimators. The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. This technique benefits from being model agnostic and can be calculated many times with different permutations of the feature*"

We prepared a small [video lecture](https://vimeo.com/745319412/1e5bd15ff7) for you to help you understand how Permutation Feature Importance works. Execute the next cell to watch it.

In [ ]:
from IPython.display import VimeoVideo

VimeoVideo("745319412?h=1e5bd15ff7", width=800, height=600)

To summarize, Permutation Feature Importance for a dataset of your choice works in the following way:

First, an already trained model predicts outputs and computes a performance measure for our dataset. This will serve as baseline performance.

Then, we carry out the following steps for each feature (potentially repeated multiple times):

1) We permute the features value across the dataset, essentially messing up any connection between the label and this feature.  
2) We predict outputs and compute the performance measure based on our dataset with the permuted feature.  
3) We compare the performance measure with the baseline performance.  
4) We use the difference between baseline performance and permuted performance as an indicator of the importance of the feature.  

If both performances are similar, the messed up feature did not lead to a decrease in model performance, indicating that the model did not rely heavily on the feature, hence assigning low importance. 
On the other hand, if the performance of the data with the permuted feature is much worse than the baseline performance, this shows that the model highly depended on the feature to produce good scores.

**Note:** this method is a **global** method which means that it only provides explanations for a full dataset, but not for individual samples.

Now lets use the scikit-learn implementation called `permutation_importance` to get some insights into the Kernel Ridge regression model we trained above. For measuring the performance drop when permuting a feature, we use the standard metric of our trained model, which is the $R^2$ score. Using the same score enables us to evaluate the performace drop in relation to the baseline performance.
20 repetitions of permutation are done for each feature to get more reliable results.

In [ ]:
scorer = get_scorer("r2")

In [ ]:
explanation_train = permutation_importance(
    pipe,
    X=X_train,
    y=y_train,
    n_repeats=20,
    random_state=seed,
    scoring=scorer
)

explanation_train["feature"] = X_train.columns

We will now plot bar histograms for visualizing the importance of each feature as obtained by the mean over the 20 repetitions. To judge variability, we plot the box plots of the feature importances as well.

In [ ]:
plot_permutation_feature_importance_with_variance(explanation_train, X_train, 'Permutation Feature Importance on Training Data for California Housing Model')

Features with the largest importance score are the ones whose score is most decreased when the feature is permuted, indicating high relevance for prediction of the target. For our California Housing model, the *median income in a block group* seems to have the highest impact on the model performance. Permuting this feature leads to a $R^2$ score decrease of > 0.8. When comparing it to the baseline $R^2$ score of 0.8, we notice that permuting the *median income* feature leads to a model with $R^2$ score < 0, i.e. a model that is worse than a constant model that always predicts the average target value disregarding the input features.

We are not restricted to obtain feature importances of the same data set that we used to train the model. Instead we could use the same approach to identify the most important features in the test set.

In [ ]:
explanation_test = permutation_importance(
    pipe,
    X=X_test,
    y=y_test,
    n_repeats=20,
    random_state=seed,
    scoring=scorer)

explanation_test["feature"] = X_test.columns

plot_permutation_feature_importance_with_variance(explanation_test, X_test, 'Permutation Feature Importance on Test Data for California Housing Model')

It seems that for both datasets, largely the same features are identified as important, which is reassuring.
**Note:** this agreement of important features between training and testing datasets is not guaranteed. In such cases, it is not straightforward to decide on the "truly important" features for the model.

### Dealing with correlated features

In the video you already learned that applying permutation importance to datasets that contain correlated features can be problematic in various ways.
This is also emphasized in the [scikit_learn documentation](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance):

"*When two features are correlated and one of the features is permuted, the model will still have access to the feature through its correlated feature. This will result in a lower importance value for both features, where they might actually be important.*"

But this is not the only effect that can occur. Below we will see an example where importance is attributed to a feature which is completely unrelated to the target variable because of its correlation to an actually important feature.

Let's create toy data of multiple features of which the first two are highly correlated but only one
of them will be used for computing our target.

In [ ]:
# create toy features
np.random.seed(seed)

corr1 = 0.95
n_vars = 5

means = np.array([i*5 for i in range(n_vars)])
cov = np.diag(np.ones(n_vars))
cov[0, 1] = corr1
cov[1, 0] = corr1

features = np.random.multivariate_normal(means, cov, size=2000)
features = pd.DataFrame(features)
features.columns = ['feature_1','feature_2','feature_3','feature_4','feature_5']

Now lets create a target variable that is computed from all features BUT feature 1, which is highly correlated to feature 2.

In [ ]:
# compute target from four features and add noise
np.random.seed(seed)
target_clean = 1.e-1 * np.exp(features.iloc[:, 1]) * 0.3 * np.sin(features.iloc[:, 2]) + 0.5 * np.sin(features.iloc[:, 3]) + 0.1 * np.cos(features.iloc[:, 4])
target = target_clean + .02 * np.random.normal(0, 1, size=len(target_clean))

As we can see in the plot below, feature 1 and feature 2 are highly correlated, while the remaining feature show no correlation amongst each other.  
In addition, from the features that were used to create the target variable, feature 2 and feature 3 correlate with the target variable.

In [ ]:
df = features.copy()
df['target'] = target
plot_correlation_matrix(df)

Let's now see how the correlation between feature 1, which was not used to compute the target variable, and feature 2 influences the Permutation Feature Importance! Therefore, we train a model on all features and calculate the feature importance. 

In [ ]:
# split data into training and test data, then train model on training data
x_train, x_test, y_train, y_test = train_test_split(features, target, random_state=seed, train_size=0.6)
pipe.fit(x_train, y_train)

# model performance on training and test data
print('Model Performance on training data: {}'.format(pipe.score(x_train, y_train)))
print('Model Performance on test data: {}'.format(pipe.score(x_test, y_test)))

# Now back to obtaining importances
explanation_train = permutation_importance(
    pipe,
    X=x_train,
    y=y_train,
    n_repeats=10,
    random_state=seed,
    scoring=scorer)

explanation_train["feature"] = x_train.columns

plot_permutation_feature_importance(explanation_train, x_train, 'Permutation Feature Importance for Full Model')

Next, we remove the highly correlated feature 1, retrain the model and calculate the feature importance. Removing feature 1 should not have a great impact on the model since it was not used to create the target variable.

In [ ]:
# drop correlated feature 1
features.drop('feature_1', axis=1, inplace=True)

# split data into training and test data, then train model on training data
x_train, x_test, y_train, y_test = train_test_split(features, target, random_state=seed, train_size=0.6)
pipe.fit(x_train, y_train)

# model performance on training and test data
print('Model Performance on training data: {}'.format(pipe.score(x_train, y_train)))
print('Model Performance on test data: {}'.format(pipe.score(x_test, y_test)))

# Now back to obtaining importances
explanation_train = permutation_importance(
    pipe,
    X=x_train,
    y=y_train,
    n_repeats=10,
    random_state=seed,
    scoring=scorer)

explanation_train["feature"] = x_train.columns

plot_permutation_feature_importance(explanation_train, x_train, 'Permutation Feature Importance for Reduced Model')

As can be seen from above, the model performance does not change when removing feature 1 from the dataset, which indicated that both models are comparable.  
However, if we look at the feature importance of the full model, we can see that the importance is split between the correlated feature 1 and feature 2. Even though the data we simulated does not depend on feature 1, the model still assigns quite high importance to this feature as well, due to its high correlation with feature 2, which can lead to wrong conclusions during model interpretation! If we compare the feature importance of the full model to the one where we removed feature 1, we can see that the importance of feature 2 increases, although the importance of the remaining features stays the same. This illustrates why you have to be careful when using Permutation Feature Importance for models that were trained on datasets with high feature correlations!


<font color='green'>

#### Question 1: What is achieved by permuting feature values?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 2: Would it also be a good idea to permute the target instead of the features?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 3: How should the output of permutation importance for a single feature be interpreted?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 4: What does a negative importance value mean?

<font color='grey'>

#### Your Answer: 



<font color='green'>

#### Question 5: Why don't we leave out a feature instead of permuting it to measure its importance?

<font color='grey'>

#### Your Answer: 
